Goal of this notebook:

- establish the infinite depth wave equation and dispersion relation
- compute a formula for the particle trajectories
- plot the result of a superposition of a wave coming from the left and a wave coming from the right to imitate the Van Dyke illustration of page 111

# Equations 

We have a solution for the surface over-pressure:

$$
p(x, z, t) = \Re \left(\exp(kz) \exp(i(kx - \omega t) \right)
$$

If we reason at fixed wavenumbers, then we can deduce the frequency from the dispersion relation:
$$
D(\omega, k) = \omega^2 - gk = 0
$$

## Pressure plot

Let's try some pure waves.

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('matplotlib')

qm_opts = hv.opts.QuadMesh(fig_size=300, aspect=2.5, cmap='seismic')

In [ ]:
x = np.linspace(-6.25, 6.25, num=400).reshape(1, -1)
z = np.linspace(0, -5, num=100).reshape(-1, 1)
X, Z = np.meshgrid(x, z)
k = 2 * np.pi / 1.
rho = 1000

def omega_of_k(k):
    return np.sqrt(9.81 * k)

p = np.real(np.exp(k * z) * np.exp(1j * (k * x - omega_of_k(k) * 0)))

hv.QuadMesh((X, Z, p)).opts(qm_opts)

Let's do something like this but with a varying value of the wavenumber.

In [ ]:
def p_of_k(k):
    p = np.real(np.exp(k * z) * np.exp(1j * (k * x - omega_of_k(k) * 0)))
    return hv.QuadMesh((X, Z, p)).opts(qm_opts)

dmap_p = hv.DynamicMap(p_of_k, kdims=['k']).redim.range(k=(1, 10))
dmap_p

We can see a different behaviour as a function of wavenumbers.

Low wavenumbers (large wavelengths) go deep, while large wavenumbers (small wavelengths) stay at the surface.

In [ ]:
dmap_p[1.] + dmap_p[8]

## Time animation 

Of course, this pressure propagates from left to right. Let's do an animation of the pressure as a function of its period.

In [ ]:
def animate_pressure(k, Nframes=10):
    omega = omega_of_k(k)
    T = 2 * np.pi / omega
    ts = np.arange(Nframes) / Nframes * T
    ps = [np.real(np.exp(k * z) * np.exp(1j * (k * x - omega_of_k(k) * t))) for t in ts]
    return {t: hv.QuadMesh((X, Z, p)).opts(qm_opts) for p,t in zip(ps, ts)}

In [ ]:
%%output holomap='scrubber'
hmap_p_lowk = hv.HoloMap(animate_pressure(1))
hmap_p_lowk

In [ ]:
%%output holomap='scrubber'
hmap_p_highk = hv.HoloMap(animate_pressure(8))
hmap_p_highk

## Displacement field 

Now the question is: what are the two displacement components due to the pressure field? We already know that the surface displacement is proportional to the value of the pressure on the surface.

In [ ]:
def surface_displacement(k):
    return hv.Curve((x, np.real(np.exp(1j * (k * x - omega_of_k(k) * 0)))))

dmap_y = hv.DynamicMap(surface_displacement, kdims=['k']).redim.range(k=dmap_p.range('k'))

In [ ]:
dmap_p * dmap_y

Now this explains the displacement at the surface. But what about inside the fluid?

In [ ]:
def show_vector_field(k, t=0, nx=30, nz=15):
    x = np.linspace(-6.25, 6.25, num=nx).reshape(1, -1)
    z = np.linspace(0, -5, num=nz).reshape(-1, 1)
    X, Z = np.meshgrid(x, z)
    U = np.real(1/(rho * omega_of_k(k) ** 2) * k * np.exp(k * z) * np.exp(1j * (k * x - omega_of_k(k) * t)))
    V = np.real(1/(rho * omega_of_k(k) ** 2) * 1j * k * np.exp(k * z) * np.exp(1j * (k * x - omega_of_k(k) * t)))

    # Convert U, V to magnitude and angle
    mag = np.sqrt(U**2 + V**2)
    angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)

    field = hv.VectorField((X, Z, angle, mag)).opts(
        fig_size=300, aspect=2.5).opts(hv.opts.VectorField(color='Magnitude', 
                                                           magnitude=hv.dim('Magnitude').norm(), 
                                                           rescale_lengths=False))
    return field 

In [ ]:
show_vector_field(.5)

Let's animate this:

In [ ]:
def animate_displacement(k, Nframes=10):
    omega = omega_of_k(k)
    T = 2 * np.pi / omega
    ts = np.arange(Nframes) / Nframes * T
    return {t: show_vector_field(k, t=t) for t in ts}

We can now plot the result for low wavenumbers.

In [ ]:
%%output holomap='scrubber'
hv.HoloMap(animate_displacement(.2, Nframes=20))

As well as for large wavenumbers.

In [ ]:
%%output holomap='scrubber'
hv.HoloMap(animate_displacement(1.))

Something that is not completely apparent in the above animation is that the trajectories of the particles are little circles. Let's try to superpose the points obtained over several frames on a still image.

In [ ]:
def show_points(k, t=0, nx=30, nz=15, viz_amp=3000, jitter=0.1):
    np.random.seed(12)
    x = np.linspace(-6.25, 6.25, num=nx).reshape(1, -1)
    z = np.linspace(0, -5, num=nz).reshape(-1, 1)
    X, Z = np.meshgrid(x, z)
    X += np.random.rand(*X.shape) * jitter
    Z += np.random.rand(*Z.shape) * jitter
    U = np.real(1/(rho * omega_of_k(k) ** 2) * k * np.exp(k * Z) * np.exp(1j * (k * X - omega_of_k(k) * t)))
    V = np.real(1/(rho * omega_of_k(k) ** 2) * 1j * k * np.exp(k * Z) * np.exp(1j * (k * X - omega_of_k(k) * t)))
    points = hv.Points(((X + U * viz_amp).flat, (Z + V * viz_amp).flat))
    return points.opts(color='black', s=5, fig_size=300, aspect=2.5)

In [ ]:
def animate_trajectory(k, Nframes=10, **kwargs):
    omega = omega_of_k(k)
    T = 2 * np.pi / omega
    ts = np.arange(Nframes) / Nframes * T
    return [show_points(k, t=t, **kwargs) for t in ts]

In [ ]:
hv.Overlay(animate_trajectory(0.2, Nframes=50, nx=15, nz=9, jitter=0.2))

What we see here (for a low wavenumber wave) are the particle displacements plotted over one period of time, and they're all little circles. This does not change for high wavenumbers, however the circle amplitude gets smaller and smaller due to depth.

In [ ]:
hv.Overlay(animate_trajectory(0.1, Nframes=50, nx=15, nz=9, jitter=0.2)) + hv.Overlay(animate_trajectory(1, Nframes=50, nx=15, nz=9, jitter=0.2))

The higher the wavenumber, the more the wave stays "at the surface".

# Two waves that oppose each other 

Let's now conclude with a recreation of a picture to be found in Milton Van Dyke's *Album of fluid motion*. We use the previous code to visualize the effect of two waves of opposite directions propagating at the same time.

We will trace the resulting pressure field, the vectors as well as the trajectories all on top of each other.

In [ ]:
def compute_fields(reflected_part, k, t=0, nx=28, nz=14):
    x = np.linspace(-6.25, 6.25, num=nx).reshape(1, -1)
    z = np.linspace(0, -5, num=nz).reshape(-1, 1)
    X, Z = np.meshgrid(x, z)
    p_to_left = np.exp(k * Z) * np.exp(1j * (k * X - omega_of_k(k) * t))
    p_to_right = reflected_part * np.exp(k * Z) * np.exp(1j * (k * X + omega_of_k(k) * t))
    p = p_to_left + p_to_right
    U = np.real(1/(rho * omega_of_k(k) ** 2) * k * p)
    V = np.real(1/(rho * omega_of_k(k) ** 2) * 1j * k * p)
    return X, Z, np.real(p), U, V

In [ ]:
X, Z, p, U, V = compute_fields(reflected_part=.8, k=0.5, t=0.1)

def make_vector_field(X, Z, U, V):
    mag = np.sqrt(U**2 + V**2)
    angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)
    field = hv.VectorField((X, Z, angle, mag)).opts(
        fig_size=300, aspect=2.5).opts(hv.opts.VectorField(magnitude=hv.dim('Magnitude').norm(), 
                                                           rescale_lengths=False))
    return field

We can now do a plot of all this information together.

In [ ]:
viz_amp = 2000
hv.QuadMesh((X, Z, p)).opts(cmap='seismic') * \
                make_vector_field(X, Z, U, V) * \
                hv.Points(((X + U * viz_amp).flat, (Z + V * viz_amp).flat)) 

How can we put the image with the trajectories as a background?

In [ ]:
def make_trajectory(k, reflected_part, Nframes=10, viz_amp=4000):
    omega = omega_of_k(k)
    T = 2 * np.pi / omega
    ts = np.arange(Nframes) / Nframes * T
    frames = {}
    for t in ts:
        X, Z, p, U, V = compute_fields(reflected_part, k, t)
        points = np.c_[(X + U * viz_amp).flat, (Z + V * viz_amp).flat]
        frames[t] = hv.Points(points)
    return frames

In [ ]:
traj = make_trajectory(.5, reflected_part=.99, Nframes=30)
hv.Overlay(list(traj.values())).opts(hv.opts.Points(s=1, color='k', show_frame=False))

Let's animate this:

In [ ]:
def animate_two_waves(k, reflected_part, Nframes=10, viz_amp=4000):
    omega = omega_of_k(k)
    T = 2 * np.pi / omega
    ts = np.arange(Nframes) / Nframes * T
    frames = {}
    circle_indices = None
    for t in ts:
        X, Z, p, U, V = compute_fields(reflected_part, k, t)
        if circle_indices is None:
            circle_indices = np.random.choice(circles.shape[0], size=100)
        frames[t] = hv.QuadMesh((X, Z, p)).opts(cmap='plasma', alpha=.2) * \
                            make_vector_field(X, Z, U, V) * \
                            hv.Points(((X + U * viz_amp).flat, (Z + V * viz_amp).flat)).opts(color='black')
    return frames

In [ ]:
%%output holomap='scrubber'
hv.HoloMap(animate_two_waves(k=.5, reflected_part=0, Nframes=20), kdims='t').opts(show_frame=False, show_title=False)

In [ ]:
%%output holomap='scrubber'
hv.HoloMap(animate_two_waves(k=.5, reflected_part=.5, Nframes=20), kdims='t').opts(show_frame=False, show_title=False)

In [ ]:
%%output holomap='scrubber'
hv.HoloMap(animate_two_waves(k=.5, reflected_part=.99, Nframes=20), kdims='t').opts(show_frame=False, show_title=False)

# But there's more ! 

They're is always more to learn. Here, my principal source of learning was the MOOC Fundamentals of waves and vibrations, for free on Coursera.
Feynman's chapter on waves is also fantastic (http://www.feynmanlectures.caltech.edu/I_51.html)

http://courses.washington.edu/mengr543/handouts/Album-Fluid-Motion-Van-Dyke.pdf